# Biol 359A  | Simple and Multiple Linear Regression
### Spring 2022, Week 4
<hr>

Objectives:
-  Run and interpret a linear regression
-  Gain intuition about MLR results
-  Introduce concepts like overfitting and test-data


In [ ]:
!git clone https://github.com/BIOL359A-FoundationsOfQBio-Spr22/week4_linearregression
!mkdir ./data
!cp week4_linearregression/data/* ./data
!cp week4_linearregression/clean_data.py ./

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import sklearn as sk
import matplotlib.pyplot as plt
import ipywidgets as widgets

from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

%matplotlib inline


TITLE_FONT = 20
LABEL_FONT = 16
TICK_FONT = 16
FIG_SIZE = (12,12)
COLORS= ["#008080","#CA562C"]

sns.set_context("notebook")
sns.set_style("whitegrid",  {'axes.linewidth': 2, 'axes.edgecolor':'black'})
sns.set(font_scale=1, rc={'figure.figsize':FIG_SIZE}) 



### We are going to use the Wisconsin Diagnostic Breast Cancer dataset once again

From the data source: Wisconsin Diagnostic Breast Cancer (WDBC)

```
	Features are computed from a digitized image of a fine needle
	aspirate (FNA) of a breast mass.  They describe
	characteristics of the cell nuclei present in the image.
	A few of the images can be found at
	http://www.cs.wisc.edu/~street/images/

	Separating plane described above was obtained using
	Multisurface Method-Tree (MSM-T) [K. P. Bennett, "Decision Tree
	Construction Via Linear Programming." Proceedings of the 4th
	Midwest Artificial Intelligence and Cognitive Science Society,
	pp. 97-101, 1992], a classification method which uses linear
	programming to construct a decision tree.  Relevant features
	were selected using an exhaustive search in the space of 1-4
	features and 1-3 separating planes.

	The actual linear program used to obtain the separating plane
	in the 3-dimensional space is that described in:
	[K. P. Bennett and O. L. Mangasarian: "Robust Linear
	Programming Discrimination of Two Linearly Inseparable Sets",
	Optimization Methods and Software 1, 1992, 23-34].
    
    Source:
    W.N. Street, W.H. Wolberg and O.L. Mangasarian 
	Nuclear feature extraction for breast tumor diagnosis.
	IS&T/SPIE 1993 International Symposium on Electronic Imaging: Science
	and Technology, volume 1905, pages 861-870, San Jose, CA, 1993.
```

What do all the column names mean?

- ID number
- Diagnosis (M = malignant, B = benign)

Ten real-valued features are computed for each cell nucleus:

- radius (mean of distances from center to points on the perimeter)
- texture (standard deviation of gray-scale values)
- perimeter
- area
- smoothness (local variation in radius lengths)
- compactness (perimeter^2 / area - 1.0)
- concavity (severity of concave portions of the contour)
- concave points (number of concave portions of the contour)
- symmetry 
- fractal dimension ("coastline approximation" - 1) - a measure of "complexity" of a 2D image.


Cateogory Distribution: 357 benign, 212 malignant

# Question 2: Characteristics of our dataset

Answer some questions about the structure of our data! 

In [2]:
import clean_data

original_cancer_dataset = clean_data.generate_clean_dataframe()
original_cancer_dataset.reset_index(inplace=True)
original_cancer_dataset.drop("ID", axis=1, inplace=True)
original_cancer_dataset

,diagnosis,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883
...,...,...,...,...,...,...,...,...,...,...,...
564,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623
565,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533
566,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648
567,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016


In [3]:
cancer_dataset = original_cancer_dataset.copy()
cancer_dataset["diagnosis"] = original_cancer_dataset['diagnosis'].replace(["M","B"], [1,0])
cancer_dataset

,diagnosis,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883
...,...,...,...,...,...,...,...,...,...,...,...
564,1,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623
565,1,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533
566,1,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648
567,1,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016


# Question 3-5: Simple Linear Regression

We will start with the scatter plots that we used to discuss correlation during Week 2, now through the perspective of using a Simple Linear Regression to characterize these relationships. We are going to introduce the concept of the __Coefficient of Determination__: $R^2$. 

$$ R^2 = 1 - \frac{SSR}{SST} $$

Where SSR is the Sum of Squares of Residual (model) and the SST is the Total Sum of Squares of the response variable. This value conceptually represents the "amount of variance in y is explained by x". This is a seperate concept to the Pearson Correlation Coefficient, $\rho$, which in the cases of a __strictly linear regression__, $R^2 = \rho^2$. 

Generally speaking, $R^2$ is used to characterize the fit of a model, where as $\rho$ is used to characterize the relationship between two variables. 

We will perform a simple linear regression between the response variable, y, and the feature, x. 

In [4]:
# Create scatter plots of the various features
def calculate_correlation(x,y):
    """calculate pearson correlation"""
    return calculate_mean((x - calculate_mean(x)).transpose() * (y - calculate_mean(y))) / np.sqrt(calculate_variance(x) * calculate_variance(y))

def calculate_mean(x):
    """get the sample mean"""
    return np.sum(x) / len(x)

def calculate_variance(x):
    """calculate variance of the dataset"""
    return calculate_mean((x - calculate_mean(x))**2)

def simple_linear_regression(x,y, ax):
    regression = linear_model.LinearRegression() 
    regression.fit(x.values.reshape(-1,1),y)
    coef_of_determination = regression.score(x.values.reshape(-1,1),y)
    x_range = np.linspace(min(x), max(x))
    plt.plot(x_range, regression.coef_*x_range + regression.intercept_, "--",)
    plt.text(1.01, 0.98, r"$\beta_1 = {0:.2f}$".format(regression.coef_[0]),
             ha='left', va='top', size =LABEL_FONT,
             transform=ax.transAxes)
    plt.text(1.01, 0.95, r"$\beta_0 = {0:.2f}$".format(regression.intercept_),
         ha='left', va='top', size =LABEL_FONT,
         transform=ax.transAxes)
    plt.text(1.01, 0.92, r"$R^2 = {0:.2f}$".format(coef_of_determination),
         ha='left', va='top', size =LABEL_FONT,
         transform=ax.transAxes)
    
@widgets.interact(x=list(cancer_dataset), y=list(cancer_dataset))    
def make_scatterplot(x="mean_radius",y="mean_area"):
    """make scatterplot with correlation value and regplot"""
    colors=["#e28743", "#1e81b0"]
    corr = calculate_correlation(cancer_dataset[x], cancer_dataset[y])
    index = int(corr > 0.5)
    plt.title(r"correlation: $\rho = ${:.3f}".format(corr), color= "grey", size=TITLE_FONT)
    df = cancer_dataset.reset_index()
    ax = sns.scatterplot(data=df, x=x, y=y, alpha=0.4, hue="diagnosis")
    simple_linear_regression(cancer_dataset[x], cancer_dataset[y], ax)


interactive(children=(Dropdown(description='x', index=1, options=('diagnosis', 'mean_radius', 'mean_texture', …

# Questions 6-9: Multiple Linear Regression 

We have other variables that we can use to explore the relationships in this dataset. We can use multiple linear regression to include more variables, where we are drawing a multi-dimensional shape, rather than simply a line, as our model. 

In [5]:
def linear_regression(df, feature_cols, response_col, standardized = False):
    """
    Use linear_model to run a linear regression using sklearn
    
    """
    X = df[feature_cols]
    y = df[response_col]
    if standardized:
        X = StandardScaler().fit_transform(X)
        y = StandardScaler().fit_transform(y.values.reshape(-1, 1))
    regression = linear_model.LinearRegression() 
    regression.fit(X,y)
    
    try:
        print('Intercept of MLR model is {0:0.2f}'.format(regression.intercept_))
    except TypeError:
        print('Intercept of MLR model is {0:0.2f}'.format(regression.intercept_[0]))
    print('Regression Coefficients: ')
    for feature, coef in zip(feature_cols, regression.coef_.flatten()):
        print(f'{feature} ~ {coef:.2f}')
    return regression.predict(X), regression.score(X,y)

def error_distribution(true, pred, hue=None):
    ax = plt.subplots(figsize=(12, 8))
    error = true-pred
    if hue is not None: 
        sns.kdeplot(error, hue=hue, shade=True, alpha=.2)
    else:
        sns.kdeplot(error, shade=True, alpha=.2)
    sns.despine()

    plt.xlabel(r'Residuals ($\epsilon$)')
    plt.title('Error', size=TITLE_FONT)
    plt.show()
    
def parity_plot(true, pred, r_squared=None, title='', hue=None):
    """
    plot true vs the predicted data
    inputs: 2 list-like (arrays) data structures
    """
    fig, ax = plt.subplots(1,1,figsize=(10, 8))
    if hue is not None:
        sns.scatterplot(true, pred, hue=hue)
    else: 
        sns.scatterplot(true, pred)
    min_value = min(min(true), min(pred))
    max_value = max(max(true), max(pred))
    plt.plot([min_value, max_value],[min_value, max_value], '--', label="parity")
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    ax.set_box_aspect(1)
    sns.despine()
    plt.text(1.01, 0.98, r"$R^2 = {0:.2f}$".format(r_squared),
         ha='left', va='top', size =LABEL_FONT,
         transform=ax.transAxes)
    plt.title('Parity Plot: {}'.format(title), size=TITLE_FONT)
    plt.legend(loc='best')
    plt.show()
    
    error_distribution(true, pred, hue)


def run_regression(feature_cols = 
                   ['mean_radius', 
                    'mean_texture', 
                    'mean_perimeter', 
                    'mean_smoothness', 
                    'mean_compactness', 
                    'mean_concavity', 
                    'mean_concave_points', 
                    'mean_symmetry', 
                    'mean_fractal_dimension'], 
                     response_col='mean_area',
                     standardized=False,
                     parity=True):
    y_pred, r_squared = linear_regression(cancer_dataset, feature_cols, response_col, standardized = standardized)
    if parity: parity_plot(cancer_dataset[response_col], y_pred.flatten(), r_squared, hue=cancer_dataset["diagnosis"])


@widgets.interact(response=list(cancer_dataset))   
def regression_wrapper(response="mean_area",
                       diagnosis=False,
                       radius=True, 
                       texture=True, 
                       perimeter=True, 
                       area=False,
                       smoothness=True, 
                       compactness=True, 
                       concavity=True, 
                       concave_points=True, 
                       symmetry=True, 
                       fractal_dimension=True):
    features = []
    if diagnosis: features.append("diagnosis")
    if radius: features.append("mean_radius")
    if texture: features.append("mean_texture")
    if perimeter: features.append("mean_perimeter")
    if area: features.append("mean_area")
    if smoothness: features.append("mean_smoothness")
    if compactness: features.append("mean_compactness")    
    if concavity: features.append("mean_concavity")
    if concave_points: features.append("mean_concave_points")
    if symmetry: features.append("mean_symmetry")
    if fractal_dimension: features.append("mean_fractal_dimension")
        

    run_regression(feature_cols=features, response_col = response)

interactive(children=(Dropdown(description='response', index=4, options=('diagnosis', 'mean_radius', 'mean_tex…

# Question 10: Normalization

We need to be careful about how we interpret parameters in relation to each other. One method that I can use to compare coefficients is to __standardize__ the features. This will effectively shift the distribution of all of our features to have a Standard Normal ($\mu=0,\sigma^2=1$) distribution. We can then compare the coefficients to each other, and interpret the magnitude within the model. 

Note: This transformation prevents you from making a direct interpretation of the coefficient as it relates to the actual value, and is now unitless. You should interpret these as being "model units". This transformation also means that the y-intercept will be zero.

In [6]:
@widgets.interact(response=list(cancer_dataset))   
def regression_wrapper(response="mean_area", 
                       radius=True, 
                       texture=True, 
                       perimeter=True, 
                       area=False,
                       smoothness=True, 
                       compactness=True, 
                       concavity=True, 
                       concave_points=True, 
                       symmetry=True, 
                       fractal_dimension=True):
    features = []
    if radius: features.append("mean_radius")
    if texture: features.append("mean_texture")
    if perimeter: features.append("mean_perimeter")
    if area: features.append("mean_area")
    if smoothness: features.append("mean_smoothness")
    if compactness: features.append("mean_compactness")    
    if concavity: features.append("mean_concavity")
    if concave_points: features.append("mean_concave_points")
    if symmetry: features.append("mean_symmetry")
    if fractal_dimension: features.append("mean_fractal_dimension")
        

    run_regression(feature_cols=features, response_col = response, standardized=True, parity=False)

interactive(children=(Dropdown(description='response', index=4, options=('diagnosis', 'mean_radius', 'mean_tex…

### You can ignore the code below and skip to the relevant example for question 11.

In [7]:
def quadratic(x):
    # Default - 2x^2 + 2
    return 2*x**2 + 2

def generate_noisy_data(function, noise_std, n=10, measurement_std=.2, initial_value=0, x_max=3, plot=True):
    """
    This function generates noisy data with a certain amount of error applied to the function response.
    The error is normally distributed around the noise_std.
    """
    x = np.linspace(0, x_max, n) 
    x_noise = np.random.normal(0, measurement_std, len(x))
    x += x_noise
    y_noise = np.random.normal(0, noise_std, len(x))
    y = function(x) + initial_value
    y += y_noise
    if plot:
        plt.plot(x, y, 'C0.', label='data')
        x_func = np.linspace(0, max(x)+measurement_std)
        y_func = function(x_func) + initial_value
        plt.plot(x_func, y_func, 'C0--', label='function')
        plt.fill_between(x_func, y_func+noise_std, y_func-noise_std,
                         alpha=0.1)          # Transparency of the fill
        plt.title(r'$ y = 2x^2 + 2$ with noise (std of {})'.format(noise_std))
        plt.legend(loc='best')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.xlim(0, max(x)+measurement_std)
        plt.show()
    return x, y


def plot_model(x, y, x_model, y_model, title = '',r_squared=None, x_test=None, y_test=None):
    """
    Plotter function.
    """
    
    plt.plot(x,y, 'o', label='data')
    if x_test is not None: plt.plot(x_test,y_test,"o", label='test')
    plt.plot(x_model, y_model, '--', label='model')
    plt.legend(loc='best')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim(0, max(x))
    plt.title(title)
    plt.text(1.01, 0.98, r"$R^2 = {0:.2f}$".format(r_squared),
         ha='left', va='top', size =LABEL_FONT,
         transform=plt.gca().transAxes)    
    plt.show()


def polynomial_feature_example(x, y, degrees=6, x_test=None, y_test=None):
    """
    Perform regularization on a polynomial feature set. 
    """
    poly_transform = PolynomialFeatures(degree=degrees, include_bias = False)
    x_poly = poly_transform.fit_transform(x.reshape(-1,1))
    
    #Regularization techniques need to be scaled in order to work properly
    x_scaler = StandardScaler().fit(x_poly)
    y_scaler = StandardScaler().fit(y.reshape(-1,1))
    x_poly_z = x_scaler.transform(x_poly)
    y_z = y_scaler.transform(y.reshape(-1,1))
    
    #Code to perform the model fitting and parameter estimation
    #Least Squares problem
    plt.suptitle('Linear Regression', fontsize=20, fontweight='bold')
    lm_poly = linear_model.LinearRegression(fit_intercept=True)
    lm_poly.fit(x_poly_z,y_z)

    x_model = np.linspace(min(x), max(x), 150).reshape(-1,1)
    x_model_transform = poly_transform.fit_transform(x_model)
    x_model_transform_z = x_scaler.transform(x_model_transform)
    
    
    y_model = lm_poly.predict(x_model_transform_z)*y_scaler.scale_ + y_scaler.mean_
    
    #********************************************************************************
    # Coefficients from scaled model can be transformed back into original units
    # This code is outside the scope of this class and can be ignored. 
    
    unscaled_coefficients = (lm_poly.coef_ * y_scaler.scale_ / x_scaler.scale_).flatten()
    
    poly_terms = [r'$({0:.3f})x ^ {{{1}}}$'.format(coef, i+1) for i, coef in enumerate(unscaled_coefficients)
                 if coef != 0]
    
    unscaled_intercept = lm_poly.intercept_*y_scaler.scale_ + y_scaler.mean_ \
                            - sum(unscaled_coefficients*x_scaler.mean_)
        
    intercept_str = r'${0:.1f} + $'.format(unscaled_intercept[0])
    title =  intercept_str + r'$+$'.join(poly_terms)
    #********************************************************************************
    r_squared = lm_poly.score(x_poly_z,y_z)
    ax = plot_model(x, y, x_model, y_model, title=title, r_squared = r_squared, x_test=x_test, y_test=y_test)


# Questions 11-12: Underdefined and Overdefined

In [8]:
@widgets.interact_manual(n=(0,100), degrees=(0,12))
def run_polynomial_experiment(n=20, degrees=7):
    x_data, y_data = generate_noisy_data(quadratic, 2, n)
    polynomial_feature_example(x_data, y_data, degrees=degrees)

interactive(children=(IntSlider(value=20, description='n'), IntSlider(value=7, description='degrees', max=12),…

# Question 13: Overfitting

In [9]:
@widgets.interact_manual(n=(0,100), degrees=(0,12))
def run_polynomial_experiment(n=20, degrees=7):
    x_data, y_data = generate_noisy_data(quadratic, 2, n)
    x_test, y_test = generate_noisy_data(quadratic, 2, 50, plot=False)
    polynomial_feature_example(x_data, y_data, degrees=degrees, x_test=x_test, y_test=y_test)

interactive(children=(IntSlider(value=20, description='n'), IntSlider(value=7, description='degrees', max=12),…

### Optional: 